In [1]:
import time
import google.auth
from google.auth import crypt, jwt
import requests

In [2]:
def generate_jwt(
    sa_keyfile="./api-gw-key.json",
    sa_email="api-gw@w4153-cloud-computing.iam.gserviceaccount.com",
    audience="https://w4153-test",
    expiry_length=3600,
):
    """Generates a signed JSON Web Token using a Google API Service Account."""

    now = int(time.time())

    # build payload
    payload = {
        "iat": now,
        # expires after 'expiry_length' seconds.
        "exp": now + expiry_length,
        # iss must match 'issuer' in the security configuration in your
        # swagger spec (e.g. service account email). It can be any string.
        "iss": sa_email,
        # aud must be either your Endpoints service name, or match the value
        # specified as the 'x-google-audience' in the OpenAPI document.
        "aud": audience,
        # sub and email should match the service account's email address
        "sub": sa_email,
        "email": sa_email,
    }

    # sign with keyfile
    signer = google.auth.crypt.RSASigner.from_service_account_file(sa_keyfile)
    jwt = google.auth.jwt.encode(signer, payload)

    return jwt


In [3]:
token = generate_jwt()
# token.decode("utf-8")

In [4]:
url="https://gw-t3-9im5mh4n.uk.gateway.dev"

In [5]:
res = requests.get(url)
print(res.status_code, res.content)

401 b'{"code":401,"message":"Jwt is missing"}\n'


In [6]:
def make_jwt_request(signed_jwt, url):
    """Makes an authorized request to the endpoint"""
    headers = {
        "Authorization": "Bearer {}".format(signed_jwt.decode("utf-8")),
        "content-type": "application/json",
    }
    response = requests.get(url, headers=headers)
    print(response.status_code, response.content)
    response.raise_for_status()

In [7]:
make_jwt_request(token, url)

200 b'{"message":"Welcome to the main feed service!"}'
